In [720]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model

In [721]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [722]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [723]:
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        g = d['gameID']
        yield u,g,d

In [724]:
answers = {}

In [725]:
# Some data structures that will be useful

In [726]:
allHours = []
for l in readJSON("train.json.gz"):
    allHours.append(l)

In [776]:
print(allHours[0])

('u70666506', 'g49368897', {'userID': 'u70666506', 'early_access': False, 'hours': 63.5, 'hours_transformed': 6.011227255423254, 'found_funny': 1, 'text': 'If you want to sit in queue for 10-20min and have 140 ping then this game is perfect for you :)', 'gameID': 'g49368897', 'user_id': '76561198030408772', 'date': '2017-05-20'})


In [727]:
dataTrain = [(u, g) for u, g, _ in allHours[:165000]]
dataValid = [(u, g, 1) for u, g, _ in allHours[165000:]]

In [728]:
##################################################
# Play prediction                                #
##################################################

In [729]:
# Any other preprocessing...
playedByUser = defaultdict(set)
allGames = set()

for u, g, _ in allHours:
    playedByUser[u].add(g)
    allGames.add(g)

negDataValid = [None] * len(dataValid)
for i in range(len(dataValid)):
    u = dataValid[i][0]
    g = random.sample(allGames - playedByUser[u], 1)[0]
    playedByUser[u].add(g)
    negDataValid[i] = (u, g, 0)

dataValid += negDataValid

/var/folders/jv/2nmp5ss10n7g89l5_hdgn2l40000gn/T/ipykernel_40287/2587455432.py:12: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  g = random.sample(allGames - playedByUser[u], 1)[0]


In [730]:
gameCount = defaultdict(int)
totalPlayed = 0

for _, game in dataTrain:
  gameCount[game] += 1
  totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

In [731]:
gamesPerUser = defaultdict(set)
usersPerGame = defaultdict(set)
for user, game in dataTrain:
    gamesPerUser[user].add(game)
    usersPerGame[game].add(user)

In [732]:
### Question 1

In [733]:
# Evaluate baseline strategy

def baseline_predict(dataValid, thresh):
  popular = set()
  count = 0
  for ic, i in mostPopular:
    count += ic
    popular.add(i)
    if count > totalPlayed * thresh: break

  predictions = [0] * len(dataValid)
  for i in range(len(dataValid)):
    _, g = dataValid[i][0], dataValid[i][1]
    predictions[i] = 1 if g in popular else 0
  
  return predictions

In [734]:
predictions = baseline_predict(dataValid, 0.5)
accuracy = sum([x == y for x, y in zip(predictions, [z[2] for z in dataValid])]) / len(dataValid)

In [735]:
answers['Q1'] = accuracy

In [736]:
assertFloat(answers['Q1'])

In [737]:
### Question 2

In [738]:
# Improved strategy


In [739]:
# Evaluate baseline strategy
thresholds = [x / 10 for x in range(1, 10)]
bestPopAccuracy, bestPopThreshold = 0, None

for thresh in thresholds:
    predictions = baseline_predict(dataValid, thresh)
    accuracy = sum([x == y for x, y in zip(predictions, [z[2] for z in dataValid])]) / len(dataValid)
    if accuracy > bestPopAccuracy:
        bestPopAccuracy = accuracy
        bestPopThreshold = thresh

In [740]:
answers['Q2'] = [bestPopAccuracy, bestPopThreshold]

In [741]:
assertFloatList(answers['Q2'], 2)

In [742]:
### Question 3/4

In [743]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0: return 0
    return numer / denom

def jaccard_predict(dataValid, thresh):
    predictions = [0] * len(dataValid)
    for i in range(len(dataValid)):
        u, g, _ = dataValid[i]
        bestSim = 0
        for g_ in gamesPerUser[u]:
            if g == g_: continue
            bestSim = max(bestSim, Jaccard(usersPerGame[g], usersPerGame[g_]))
        predictions[i] = 1 if bestSim > thresh else 0
    
    return predictions

def jaccard_pop_predict(dataValid, threshJac, threshPop):
    popular = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        popular.add(i)
        if count > totalPlayed * threshPop: break

    predictions = [0] * len(dataValid)
    for i in range(len(dataValid)):
        u, g, _ = dataValid[i]
        if g not in popular: continue
        bestSim = 0
        for g_ in gamesPerUser[u]:
            if g == g_: continue
            bestSim = max(bestSim, Jaccard(usersPerGame[g], usersPerGame[g_]))
        predictions[i] = 1 if bestSim > threshJac else 0
    
    return predictions

In [744]:
thresholds = [x / 100 for x in range(1, 6)]
bestJacAccuracy, bestJacThreshold = 0, None

for thresh in thresholds:
    predictions = jaccard_predict(dataValid, thresh)
    accuracy = sum([x == y for x, y in zip(predictions, [z[2] for z in dataValid])]) / len(dataValid)
    if accuracy > bestJacAccuracy:
        bestJacAccuracy = accuracy
        bestJacThreshold = thresh
print(bestJacAccuracy, bestJacThreshold)

0.6744174417441744 0.03


In [745]:
predictions = jaccard_predict(dataValid, bestJacThreshold)
q3_accuracy = sum([x == y for x, y in zip(predictions, [z[2] for z in dataValid])]) / len(dataValid)

predictions = jaccard_pop_predict(dataValid, bestJacThreshold, bestPopThreshold)
q4_accuracy = sum([x == y for x, y in zip(predictions, [z[2] for z in dataValid])]) / len(dataValid)

answers['Q3'] = q3_accuracy
answers['Q4'] = q4_accuracy

In [746]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [747]:
### Question 5

In [748]:
predictions = open("predictions_Played.csv", 'w')
for l in open("pairs_Played.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u, g = l.strip().split(',')
    
    bestSim = 0
    for g_ in gamesPerUser[u]:
        if g == g_: continue
        bestSim = max(bestSim, Jaccard(usersPerGame[g], usersPerGame[g_]))
    pred = 1 if bestSim > bestJacThreshold else 0
    
    _ = predictions.write(u + ',' + g + ',' + str(pred) + '\n')

predictions.close()

In [749]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [750]:
##################################################
# Hours played prediction                        #
##################################################

In [751]:
hoursTrain = [(x, y, z['hours_transformed']) for (x, y, z) in allHours[:165000]]
hoursValid = [(x, y, z['hours_transformed']) for (x, y, z) in allHours[165000:]]
trainHours = [h for _, _, h in hoursTrain]
globalAverage = sum(trainHours) / len(trainHours)

In [752]:
hours = defaultdict()
users, items = set(), set()
usersPerItem, itemsPerUser = defaultdict(set), defaultdict(set)

for u, i, h in hoursTrain:
    hours[(u, i)] = h
    users.add(u)
    items.add(i)
    usersPerItem[i].add(u)
    itemsPerUser[u].add(i)

In [785]:
### Question 6

In [786]:
alpha = globalAverage
betaU = {u: 0 for u in users}
betaI = {i: 0 for i in items}

In [787]:
def iterate(alpha, betaU, betaI, lamb):
    alpha = sum([h - betaU[u] - betaI[i] for u, i, h in hoursTrain]) / len(hoursTrain)

    for u in users:
        betaU[u] = sum([hours[(u, i)] - alpha - betaI[i] for i in itemsPerUser[u]]) / (lamb + len(itemsPerUser[u]))
    
    for i in items:
        betaI[i] = sum([hours[(u, i)] - alpha - betaU[u] for u in usersPerItem[i]]) / (lamb + len(usersPerItem[i]))
    
    return alpha, betaU, betaI


In [790]:
currMSE = float('inf')
MSE_THRESH = 0.1
LAMB = 0.01

while True:
    alpha, betaU, betaI = iterate(alpha, betaU, betaI, LAMB)
    res = sum([(alpha + betaU[u] + betaI[i] - h)**2 for u, i, h in hoursTrain])
    res += LAMB * (sum([betaU[u]**2 for u in betaU]) + sum([betaI[i]**2 for i in betaI]))
    res /= len(hoursTrain)
    if currMSE - res < MSE_THRESH: break
    else: currMSE = res

In [791]:
validMSE = sum([(alpha + betaU[u] + betaI[i] - h)**2 for u, i, h in hoursValid]) / len(hoursValid)
answers['Q6'] = validMSE

In [792]:
assertFloat(answers['Q6'])

In [793]:
### Question 7

In [794]:
betaUs = [(betaU[u], u) for u in betaU]
betaIs = [(betaI[i], i) for i in betaI]
betaUs.sort()
betaIs.sort()

print("Maximum betaU = " + str(betaUs[-1][1]) + ' (' + str(betaUs[-1][0]) + ')')
print("Maximum betaI = " + str(betaIs[-1][1]) + ' (' + str(betaIs[-1][0]) + ')')
print("Minimum betaU = " + str(betaUs[0][1]) + ' (' + str(betaUs[0][0]) + ')')
print("Minimum betaI = " + str(betaIs[0][1]) + ' (' + str(betaIs[0][0]) + ')')

Maximum betaU = u60898505 (6.291149718277626)
Maximum betaI = g17604638 (4.95978077913233)
Minimum betaU = u13037838 (-3.0493938535181924)
Minimum betaI = g84397720 (-3.44310863380465)


In [795]:
answers['Q7'] = [betaUs[-1][0], betaUs[0][0], betaIs[-1][0], betaIs[0][0]]

In [796]:
answers['Q7']

[6.291149718277626, -3.0493938535181924, 4.95978077913233, -3.44310863380465]

In [763]:
assertFloatList(answers['Q7'], 4)

In [764]:
### Question 8

In [765]:
# Better lambda...
lambdas = [0.0001, 0.001, 0.01, 0.1, 1, 10] 
bestLambda, bestMSE = None, float('inf')

for lamb in lambdas:
    currMSE = float('inf')
    while True:
        alpha, betaU, betaI = iterate(alpha, betaU, betaI, lamb)
        res = sum([(alpha + betaU[u] + betaI[i] - h)**2 for u, i, h in hoursTrain])
        res += lamb * (sum([betaU[u]**2 for u in betaU]) + sum([betaI[i]**2 for i in betaI]))
        res /= len(hoursTrain)
        if currMSE - res < MSE_THRESH: break
        else: currMSE = res
    if currMSE < bestMSE:
        bestMSE = currMSE
        bestLambda = lamb

In [766]:
answers['Q8'] = (bestLambda, bestMSE)

In [767]:
assertFloatList(answers['Q8'], 2)

In [768]:
LAMB = bestLambda
currMSE = float('inf')
while True:
    alpha, betaU, betaI = iterate(alpha, betaU, betaI, lamb)
    res = sum([(alpha + betaU[u] + betaI[i] - h)**2 for u, i, h in hoursTrain])
    res += lamb * (sum([betaU[u]**2 for u in betaU]) + sum([betaI[i]**2 for i in betaI]))
    res /= len(hoursTrain)
    if currMSE - res < MSE_THRESH: break
    else: currMSE = res

In [784]:
predictions = open("predictions_Hours.csv", 'w')
user_pred = set()
game_pred = set()
for l in open("pairs_Hours.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,g = l.strip().split(',')
    
    bu = betaU[u] if u in betaU else 0
    bi = betaI[g] if g in betaI else 0
    user_pred.add(u)
    game_pred.add(g)
    
    _ = predictions.write(u + ',' + g + ',' + str(alpha + bu + bi) + '\n')

predictions.close()

In [770]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()